# Booking.com Web Scraping

This notebook contains the codebase to scrape Booking.com hotel data and create datasets containing the following information:
>- A method to scrape booking.com data using beautiful soup
>- An aggregated dataset contained information about the hotels such as Hotel name, address, location coordinates, price lists, room types and the url of the specific hotels
>- Specific datasets for prices containing the maximum, minimum and average price
>- Specific datasets for room types

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import statistics 

## Initializing Urls

In [2]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.67'}
url1 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.en-gb.html%3Faid%3D331424%3Bsid%3D1a6c6cf39050b85a890e92199f697415%3Btmpl%3Dsearchresults%3Bcheckin_month%3D1%3Bcheckin_monthday%3D1%3Bcheckin_year%3D2022%3Bcheckout_month%3D1%3Bcheckout_monthday%3D7%3Bcheckout_year%3D2022%3Bclass_interval%3D1%3Bdest_id%3D102%3Bdest_type%3Dcountry%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcountry%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3De0a16cab5b24005f%3Bss%3DIreland%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DIreland%3Bssne_untouched%3DIreland%3Btop_ufis%3D1%26%3B&ss=Ireland&is_ski_area=0&ssne=Ireland&ssne_untouched=Ireland&dest_id=102&dest_type=country&checkin_year=2021&checkin_month=10&checkin_monthday=8&checkout_year=2021&checkout_month=10&checkout_monthday=13&group_adults=2&group_children=0&no_rooms=1&sb_changed_dates=1&from_sf=1'
url2 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=c3706cb4ee4a0021&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=25'
url3 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=a6c16d3fa93d010b&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=50'
url4 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=560c6d4ccd360234&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=75'
url5 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=03486d5533f101aa&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=100'
url6 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=18086d5cfdf7008c&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=125'
url7 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=35806d63485000b5&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=150'
url8 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=48176d6afa170130&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=175'
url9 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=bf326d71f3530200&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=200'
url10 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=d7436d7905080072&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=225'
url11 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=cec86d822411016f&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=250'
url12 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=3d766d896d4000ba&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=275'
url13 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=2a1b6d905bcf01e9&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=300'
url14 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=7d206d98c81a015b&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=325'
url15 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=a7946da332c300b6&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=350'
url16 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=797d6db68c9d01b4&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=375'
url17 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=d4606dbda1e80061&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=400'
url18 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=1d526dc4c0070153&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=425'
url19 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=485b6e1915be0096&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=450'
url20 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=1d2e6e227e5c0137&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=475'
url21 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=88466e2a2af600be&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=500'
url22 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=eb8c6e6c35220196&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=525'
url23 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=592e6e74149c0018&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=550'
url24 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=70b86e7c9a5c0051&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=575'
url25 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=b17d6e88d68c0248&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=600'
url26 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=f9746e98f14b00a1&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=625'
url27 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=ca5e6e9f568e01cb&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=650'
url28 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=287e6ea9f0e7012c&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=675'
url29 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=93d06eb2ba220184&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=700'
url30 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=eee16ecc45c20025&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=725'
url31 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=44966ed5197800ca&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=750'
url32 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=67916edd85e9019c&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=775'
url33 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=bbff6ee53a4901bb&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=800'
url34 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=39146efd406a0168&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=825'
url35 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=eee16f09444b0100&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=850'
url36 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=290b6f149e630096&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=875'
url37 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=32be6f28417800e0&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=900'
url38 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=f2836f351e6c012c&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=925'
url39 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&from_sf=1&group_adults=2&group_children=0&label_click=undef&no_rooms=1&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=1aec6f3e98130034&ss=Ireland&ssb=empty&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=950'
url40 = 'https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&tmpl=searchresults&checkin_month=10&checkin_monthday=8&checkin_year=2021&checkout_month=10&checkout_monthday=13&checkout_year=2021&class_interval=1&dest_id=102&dest_type=country&dtdisc=0&from_sf=1&group_adults=2&group_children=0&inac=0&index_postcard=0&label_click=undef&no_rooms=1&postcard=0&raw_dest_type=country&room1=A%2CA&sb_price_type=total&shw_aparth=1&slp_r_match=0&src=searchresults&src_elem=sb&srpvid=5fe46f56040a0005&ss=Ireland&ss_all=0&ssb=empty&sshis=0&ssne=Ireland&ssne_untouched=Ireland&top_ufis=1&rows=25&offset=975'

urls = [url1, url2, url3, url4, url5, url6, url7, url8, url9, url10, url11, url12, url13, url14, url15, url16, url17, url18, url19, url20, url21, url22, url23, url24, url25, url26, url27, url28, url29, url30, url31, url32, url33, url34, url35, url36, url37, url38, url39, url40]

## Defining the web scraper function

In [26]:
#Function to scrape booking.com
def web_scraper(url):
    
    #requesting access to the website from server
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'lxml')
    
    #selecting the block for property data
    soup.select('.sr_property_block')
    hotel_data = []
    for item in soup.select('.sr_property_block'):
        try: 
            #selecting hotel urls
            all_links = item.find_all("a", class_="js-sr-hotel-link hotel_name_link url")
            #looping through each hotel url
            for link in all_links:
                hotel_url = link.get("href")
                new_url = re.sub(r"\s+", "", hotel_url)
                prefix = "https://booking.com"
                booking = prefix+new_url
                #hotel url; aid is a cookie and could be replaced for ?
                res = booking.split('aid')[0]
                
                #requesting each hotel url
                response = requests.get(booking, headers=headers)
                soup_link = BeautifulSoup(response.content, 'lxml')
                
                #initialising room type and price lists
                room_types = []
                prices = []
                
                #scraping and appending room types
                for item_1 in soup_link.select('.hprt-roomtype-link'):
                    room = item_1.select('.hprt-roomtype-icon-link')[0].get_text().strip()
                    room_types.append(room)
                
                #scraping and appending prices
                for item_2 in soup_link.select('.d_pd_hp_price_left_align'):
                    price = item_2.select('.bui-u-sr-only')[0].get_text().strip()
                    price = price.replace('Price', '').strip()
                    #replacing the UTF encoding for €
                    price = price.replace(u'\xa0', u'') 
                    price = price.replace('€','')
                    price = price.replace(',','')

                    prices.append(price.strip())

                #preprocessing string
                rooms = " ".join(str(elem) for elem in room_types)
                price_new = " ".join(str(elem) for elem in prices)
                
                price_list = list(map(int, prices))
#                 average = sum(mylist) / len(mylist)
                average = round((sum(price_list) / len(price_list)),2)
                co_ords = item.select('.bui-link')[0].get("data-coords").strip()
                long, lat = co_ords.split(',')
                #aggregating the dataframe
                hotel_data.append(
                {
                    'Hotel': item.select('.sr-hotel__name')[0].get_text().strip(),
                    'Address':soup_link.select('.hp_address_subtitle')[0].get_text() ,
                    'Rating': item.select('.bui-review-score__badge')[0].get_text().strip(),
                    'Review': item.select('.bui-review-score__title')[0].get_text().strip(),
                    'Room Types': rooms,
                    'Prices': price_new,
                    'Maximum Price': max(price_list),
                    'Minimum Price': min(price_list),
                    'Average': average,
                    'Coordinates': co_ords,
                    'Lattitude': lat,
                    'Longitude': long,
                    'Specific Link': booking,
                    'Hotel URL': res         
                }
                    )
   
        except Exception as e:
            print('')
    hotel_data = pd.DataFrame(hotel_data)
    print(hotel_data)
    return hotel_data
    

In [27]:
d1 = web_scraper(url1)

                                              Hotel  \
0                                River Island Hotel   
1                        The Yeats County Inn Hotel   
2                                 Joyce's Inishowen   
3                                Downings Bay Hotel   
4   Connemara National Park lodge Letterfrack Lodge   
5                               The Highlands Hotel   
6                        The Skellig Lodge & Hostel   
7                                      Marine Hotel   
8                                     Armcashel B&B   
9                           Dorrians Imperial Hotel   
10                   Macreddin Rock Bed & Breakfast   
11                       GN Ballykisteen Golf Hotel   
12                                    Hotel Newport   
13                               Klondyke House B&B   
14                    The Valley House Hostel & Bar   
15                                  Germaines Hotel   
16                          The Listowel Arms Hotel   
17        

In [28]:
d1

,Hotel,Address,Rating,Review,Room Types,Prices,Maximum Price,Minimum Price,Average,Coordinates,Lattitude,Longitude,Specific Link,Hotel URL
0,River Island Hotel,"\nMain Street, Castleisland, Ireland\n",8.4,Very good,Double Room Twin Room,423 423,423,423,423.00,"-9.46415305137634,52.2307286420629",52.2307286420629,-9.46415305137634,https://booking.com/hotel/ie/riverisland.en-gb...,https://booking.com/hotel/ie/riverisland.en-gb...
1,The Yeats County Inn Hotel,"\nCurry , Tobercurry, Ireland\n",8.3,Very good,Double Room Standard Family Room Triple Room S...,395 475 235 275 395 475 530 650 235 275 395 47...,650,235,389.44,"-8.77097263973997,54.0033208979175",54.0033208979175,-8.77097263973997,https://booking.com/hotel/ie/the-yeats-country...,https://booking.com/hotel/ie/the-yeats-country...
2,Joyce's Inishowen,"\nCarndonagh The Diamond, Carndonagh, Ireland\n",9.3,Superb,Standard Double Room Double or Twin Room,333 428,428,333,380.50,"-7.261806,55.251044",55.251044,-7.261806,https://booking.com/hotel/ie/joyce-39-s-inisho...,https://booking.com/hotel/ie/joyce-39-s-inisho...
3,Downings Bay Hotel,"\nMain Street, . Downings, Ireland\n",8.8,Fabulous,Twin Room Family Room,660 611 810 751,810,611,708.00,"-7.83601999282837,55.1946337460849",55.1946337460849,-7.83601999282837,https://booking.com/hotel/ie/downings-bay.en-g...,https://booking.com/hotel/ie/downings-bay.en-g...
4,Connemara National Park lodge Letterfrack Lodge,"\nConnemara, Letterfrack, Ireland\n",8.6,Fabulous,Double Room Triple Room Quadruple Room,350 375 450,450,350,391.67,"-9.95072597230319,53.5543666346719",53.5543666346719,-9.95072597230319,https://booking.com/hotel/ie/letterfrack-lodge...,https://booking.com/hotel/ie/letterfrack-lodge...
5,The Highlands Hotel,"\nHighlands Hotel, Main Street, Glenties, Irel...",8.9,Fabulous,Deluxe Double Room Deluxe Double Room (2 Adult...,630 750,750,630,690.00,"-8.28127473592758,54.7957458523372",54.7957458523372,-8.28127473592758,https://booking.com/hotel/ie/the-highlands.en-...,https://booking.com/hotel/ie/the-highlands.en-...
6,The Skellig Lodge & Hostel,"\nBallinskelligs, Co. Kerry, Ballinskelligs, I...",8.7,Fabulous,Standard Double Room with Shared Bathroom Stan...,285 285 225 320 425,425,225,308.00,"-10.283031463623,51.8235775215815",51.8235775215815,-10.283031463623,https://booking.com/hotel/ie/skellig-hostel.en...,https://booking.com/hotel/ie/skellig-hostel.en...
7,Marine Hotel,"\nSutton Cross, D13 Sutton, Ireland\n",8.1,Very good,Double Room Twin Room Deluxe Double Room with ...,795 645 795 645 895 745,895,645,753.33,"-6.11034035682678,53.3888442110892",53.3888442110892,-6.11034035682678,https://booking.com/hotel/ie/marinehoteldublin...,https://booking.com/hotel/ie/marinehoteldublin...
8,Armcashel B&B,"\nKnock Road, Castlerea, Ireland\n",9.1,Superb,Double Room,400 275,400,275,337.50,"-8.51211905479431,53.7693626261669",53.7693626261669,-8.51211905479431,https://booking.com/hotel/ie/armcashel-b-amp-b...,https://booking.com/hotel/ie/armcashel-b-amp-b...
9,Dorrians Imperial Hotel,"\nMain Street, Ballyshannon, Ireland\n",8.7,Fabulous,Double Room Twin Room,550 550,550,550,550.00,"-8.19150924682617,54.502097028033",54.502097028033,-8.19150924682617,https://booking.com/hotel/ie/dorrians-imperial...,https://booking.com/hotel/ie/dorrians-imperial...


## Creating an aggregated dataset and cleaning the dataset

In [29]:
#scraping and aggregating data for 40 urls
df_all = pd.DataFrame([])
print(urls)

for url in urls:
    df = web_scraper(url)
    df_all = df_all.append(df, ignore_index=True)

['https://www.booking.com/searchresults.en-gb.html?aid=331424&sid=1a6c6cf39050b85a890e92199f697415&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.en-gb.html%3Faid%3D331424%3Bsid%3D1a6c6cf39050b85a890e92199f697415%3Btmpl%3Dsearchresults%3Bcheckin_month%3D1%3Bcheckin_monthday%3D1%3Bcheckin_year%3D2022%3Bcheckout_month%3D1%3Bcheckout_monthday%3D7%3Bcheckout_year%3D2022%3Bclass_interval%3D1%3Bdest_id%3D102%3Bdest_type%3Dcountry%3Bdtdisc%3D0%3Bfrom_sf%3D1%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcountry%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3De0a16cab5b24005f%3Bss%3DIreland%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3DIreland%3Bssne_untouched%3DIreland%3Btop_ufis%3D1%26%3B&ss=Ireland&is_ski_area=0&ssne=Ireland&ssne_untouched=Ireland&dest_id=102&des










                                                Hotel  \
0                     Jurys Inn Dublin Parnell Street   
1                                    The Gibson Hotel   
2   Holiday Inn Express Dublin City Centre, an IHG...   
3                             The Galmont Hotel & Spa   
4                                    Flannery's Hotel   
5                                Clayton Hotel Galway   
6   The Address Connolly (formerly The North Star ...   
7                              Sheraton Athlone Hotel   
8                                      Jurys Inn Cork   
9             Killarney Towers Hotel & Leisure Centre   
10              Hilton Garden Inn Dublin Custom House   
11                 Maldron Hotel South Mall Cork City   
12                                     Zanzibar Locke   
13                      Jurys Inn Dublin Christchurch   
14    The Marker Hotel - A Leading Hotel of the World   
15                       Maldron Hotel Dublin Airport   

                     

                                    Hotel  \
0                     The Glenbeigh Hotel   
1                      Dingle Gate Hostel   
2                             Kee's Hotel   
3                    Lakeside Manor Hotel   
4                         West Cork Hotel   
5                       The Auld Triangle   
6   Schull Harbour Hotel & Leisure Centre   
7               Harbour House Guest House   
8                     Vaughans Anchor Inn   
9                       The Dolphin Hotel   
10              Careys Viking House Hotel   
11                    Bracken Court Hotel   
12                   Castle Hotel Macroom   
13               Fallon's Bed & Breakfast   
14                       The Bianconi Inn   
15                      The Gateway Hotel   
16                             Tara Hotel   
17                  Templemore Arms Hotel   
18                     Glenview Lodge B&B   
19                   The Inn at Dromoland   
20                    Rockville House B&B   
21        





                                                Hotel  \
0                                      The Pier Hotel   
1                             Slieve League House B&B   
2                                        Beds of Silk   
3                             East Clare Golf Village   
4                                   Jacobs Inn Hostel   
5                                     Annie May's B&B   
6                                Clancys Of Glenfarne   
7                                   Ballycannon Lodge   
8                                     Hotel Killarney   
9                                      Clooneen House   
10                                   The Gibson Hotel   
11                 Maldron Hotel South Mall Cork City   
12  Holiday Inn Express Dublin City Centre, an IHG...   
13                                     Temple Bar Inn   
14                                The Newgrange Hotel   
15                             Sheraton Athlone Hotel   
16                         



                                                Hotel  \
0      Springhill Court Hotel & Leisure Club Kilkenny   
1                               Killarney Court Hotel   
2                     Maldron Hotel Sandy Road Galway   
3                                Hazelbrook House B&B   
4                           Greenfields Country House   
5                                      Reenglas House   
6          Holiday Inn - Dublin Airport, an IHG Hotel   
7                               Murphys Farmhouse B&B   
8             Maldron Hotel & Leisure Centre Limerick   
9               Hilton Garden Inn Dublin Custom House   
10           Travelodge Dublin Airport North 'Swords'   
11                                      Arthaus Hotel   
12                           Clayton Hotel Charlemont   
13               Coral Gables Guesthouse & Campervans   
14                             Bellbridge House Hotel   
15                              My Place Dublin Hotel   
16                           

...

                                         Hotel  \
0         Motorwaylodge Kilcoran Cahir E21RK60   
1                                   Rheban BnB   
2                          The Stone House B&B   
3                          Ballina Manor Hotel   
4                            Ardnagashel Woods   
5                       Waterford Viking Hotel   
6                           Atlantic House B&B   
7                             Tralee Townhouse   
8                                  Tranquility   
9                                An Bothar Pub   
10                      St. Aiden's Guesthouse   
11                               Valentia View   
12                              Hawthorn Lodge   
13                               Country House   
14                           Dun Ri Guesthouse   
15                             Rockcrest House   
16                          Grand Hotel Tralee   
17                                   The Lodge   
18                         Kinlay House Dublin   


                                              Hotel  \
0          Tralee Holiday Lodge Guest Accommodation   
1                               Seagull Cottage B&B   
2                             Westbrook House B & B   
3                      Sleepzone Hostel Galway City   
4                         Town Square Holiday Homes   
5                                      Bythesea B&B   
6                                  Old Ground Hotel   
7                             Clifden House by Nina   
8                        Beach View Bed & Breakfast   
9                          The Hoban Hotel Kilkenny   
10                                  The Rainbow POD   
11                                Alpine Guesthouse   
12                           The Farnham Arms Hotel   
13                                Country Home Tara   
14                              Creggan Court Hotel   
15                                  The Dean Dublin   
16                              Wild Atlantic Stays   
17        




                                 Hotel  \
0                   Magpie Bar and B&B   
1         Brú na Dromoda/Dromid Hostel   
2                  Brogans Bar & Hotel   
3                  Midlands Park Hotel   
4     Westlodge Hotel & Leisure Centre   
5   the Continental Boutique Residence   
6                 Little Glebe Cottage   
7            Inch Hideaway Eco Camping   
8                      Relaxing space.   
9                            Sandawana   
10     McSorleys Accommodation and Bar   
11                          Aran Lodge   
12                 Midleton Park Hotel   
13      Kilcommon Lodge Holiday Hostel   
14                       Killarney Inn   
15                     An Charraig Ban   
16                   copperfield house   
17            Barron's Bed & Breakfast   
18            Slievemore Holiday Homes   
19                       Killard House   
20                   Corrib View Lodge   
21             The Meadows Guest House   

                              

                                                Hotel  \
0                                  Ballinwillin House   
1                                       Drumhouse B&B   
2                                      The Ferry Boat   
3                             Killilagh Accommodation   
4                                        Barrow House   
5   Bradog House - Beautiful 4 Bedroom House in Ce...   
6                                   Butlers Townhouse   
7                                        The Longboat   
8                                       Cois Farraige   
9                       Waterford Castle Hotel Lodges   
10                                  The Gateway Hotel   
11                                    Dungimmon House   
12                  Gardenfield House Bed & Breakfast   
13                                    Slieve Elva B&B   
14                                 Klondyke House B&B   
15                                Ben Breen House B&B   
16                             




                                   Hotel  \
0                The Meadows Guest House   
1     big bright room and happy box room   
2                       Abigail's Hostel   
3            Riverside Marina Apartments   
4           Sea Breeze Bed and Breakfast   
5              Cloon River Self-Catering   
6                     The Parkavon Hotel   
7                   Atlantic Guest House   
8                    Brandon House Hotel   
9           Redcastle Chateau by the Sea   
10                   Station View Tavern   
11                   Roxford Lodge Hotel   
12                      Divine Mercy B&B   
13                           Tatler Jack   
14                 Bank House Guesthouse   
15  Bruckless Rest - Fine Country Living   
16                       Abbeyview House   
17                       The White House   
18          The Villa Rose Hotel & V-Spa   
19                          Neidin House   
20                Hillview Self Catering   
21                      Westp

In [30]:
#splitting coordinates into lattitude and longitutde
df_all.drop_duplicates()
df_all=df_all.replace('\n','',regex=True).astype(str)
df_all.to_csv('Booking.com_Full_Dataset.csv', index=False)
df_all

,Hotel,Address,Rating,Review,Room Types,Prices,Maximum Price,Minimum Price,Average,Coordinates,Lattitude,Longitude,Specific Link,Hotel URL
0,Jurys Inn Dublin Parnell Street,"Moore Street Plaza, Parnell Street, D1 Dublin,...",8.1,Very good,Standard Twin Room Superior Twin Room Superior...,558 686 697 807 639 752 598 747 857 802 598 74...,1012,558,750.18,"-6.2629097700119,53.3516304175292",53.3516304175292,-6.2629097700119,https://booking.com/hotel/ie/jurysinnparnellst...,https://booking.com/hotel/ie/jurysinnparnellst...
1,The Gibson Hotel,"The Point Village, D1 Dublin, Ireland",8.6,Fabulous,Superior Double Room Superior Twin Room Double...,705 835 770 705 835 770 805 935 870 880 1010 9...,1275,705,925.26,"-6.22855871915817,53.3485371404209",53.3485371404209,-6.22855871915817,https://booking.com/hotel/ie/the-gibson.en-gb....,https://booking.com/hotel/ie/the-gibson.en-gb....
2,"Holiday Inn Express Dublin City Centre, an IHG...","28-32 O'Connell Street Upper, D01 T2X2 Dublin,...",8.8,Fabulous,Double Room Room allocated on arrival,554 615 526 554 615 526,615,526,565.0,"-6.26069316931148,53.3521065483007",53.3521065483007,-6.26069316931148,https://booking.com/hotel/ie/holiday-inn-expre...,https://booking.com/hotel/ie/holiday-inn-expre...
3,The Galmont Hotel & Spa,"Lough Atalia Road, Galway, Ireland",8.5,Very good,Superior King Room King Room - Disability Acce...,875 975 875 975 1050 1150 1925 2025 2075,2075,875,1325.0,"-9.04308915138245,53.2741766017925",53.2741766017925,-9.04308915138245,https://booking.com/hotel/ie/radisson-sas-spa-...,https://booking.com/hotel/ie/radisson-sas-spa-...
4,Flannery's Hotel,"Dublin Road, . Galway, Ireland",8.4,Very good,Double Room Twin Room Family Room Superior Kin...,551 646 551 646 616 711 631 726 656 751 696 79...,1263,483,742.4,"-9.01465237140656,53.2781092441103",53.2781092441103,-9.01465237140656,https://booking.com/hotel/ie/flanneryshotel.en...,https://booking.com/hotel/ie/flanneryshotel.en...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,The White House,"Pearse Street, Kinsale, Ireland",8.7,Fabulous,Double Room Twin Room,915 915,915,915,915.0,"-8.52333873510361,51.7065403158146",51.7065403158146,-8.52333873510361,https://booking.com/hotel/ie/the-white-house.e...,https://booking.com/hotel/ie/the-white-house.e...
893,The Villa Rose Hotel & V-Spa,"Main Street, Ballybofey, Ireland",8.9,Fabulous,Double or Twin Room,595,595,595,595.0,"-7.78147995471954,54.7995469321867",54.7995469321867,-7.78147995471954,https://booking.com/hotel/ie/the-villa-rose-v-...,https://booking.com/hotel/ie/the-villa-rose-v-...
894,Neidin House,"Landsdown Lodge, Kenmare, Ireland",9.0,Superb,Double or Twin Room Triple Room,400 500,500,400,450.0,"-9.57696944475174,51.880729836645",51.880729836645,-9.57696944475174,https://booking.com/hotel/ie/neidin-house.en-g...,https://booking.com/hotel/ie/neidin-house.en-g...
895,Hillview Self Catering,"Ballaghstown, Lusk, Lusk, Ireland",9.6,Exceptional,Deluxe Two-Bedroom Apartment,1050,1050,1050,1050.0,"-6.14556312561035,53.5423826268545",53.5423826268545,-6.14556312561035,https://booking.com/hotel/ie/hillview-self-cat...,https://booking.com/hotel/ie/hillview-self-cat...


In [31]:
#Splitting df into multiple columns
df_price = df_all[['Hotel','Maximum Price', 'Minimum Price', 'Average']]
df_price.to_csv('Booking_Price_Dataset.csv', index=False)
df_price

,Hotel,Maximum Price,Minimum Price,Average
0,Jurys Inn Dublin Parnell Street,1012,558,750.18
1,The Gibson Hotel,1275,705,925.26
2,"Holiday Inn Express Dublin City Centre, an IHG...",615,526,565.0
3,The Galmont Hotel & Spa,2075,875,1325.0
4,Flannery's Hotel,1263,483,742.4
...,...,...,...,...
892,The White House,915,915,915.0
893,The Villa Rose Hotel & V-Spa,595,595,595.0
894,Neidin House,500,400,450.0
895,Hillview Self Catering,1050,1050,1050.0


In [33]:
df_SR = df_all[['Hotel', 'Address', 'Rating', 'Review', 'Coordinates', 'Lattitude', 'Longitude', 'Specific Link']]

df_SR.to_csv('Booking.com_SR_Dataset.csv', index=False)
  
df_SR

,Hotel,Address,Rating,Review,Coordinates,Lattitude,Longitude,Specific Link
0,Jurys Inn Dublin Parnell Street,"Moore Street Plaza, Parnell Street, D1 Dublin,...",8.1,Very good,"-6.2629097700119,53.3516304175292",53.3516304175292,-6.2629097700119,https://booking.com/hotel/ie/jurysinnparnellst...
1,The Gibson Hotel,"The Point Village, D1 Dublin, Ireland",8.6,Fabulous,"-6.22855871915817,53.3485371404209",53.3485371404209,-6.22855871915817,https://booking.com/hotel/ie/the-gibson.en-gb....
2,"Holiday Inn Express Dublin City Centre, an IHG...","28-32 O'Connell Street Upper, D01 T2X2 Dublin,...",8.8,Fabulous,"-6.26069316931148,53.3521065483007",53.3521065483007,-6.26069316931148,https://booking.com/hotel/ie/holiday-inn-expre...
3,The Galmont Hotel & Spa,"Lough Atalia Road, Galway, Ireland",8.5,Very good,"-9.04308915138245,53.2741766017925",53.2741766017925,-9.04308915138245,https://booking.com/hotel/ie/radisson-sas-spa-...
4,Flannery's Hotel,"Dublin Road, . Galway, Ireland",8.4,Very good,"-9.01465237140656,53.2781092441103",53.2781092441103,-9.01465237140656,https://booking.com/hotel/ie/flanneryshotel.en...
...,...,...,...,...,...,...,...,...
892,The White House,"Pearse Street, Kinsale, Ireland",8.7,Fabulous,"-8.52333873510361,51.7065403158146",51.7065403158146,-8.52333873510361,https://booking.com/hotel/ie/the-white-house.e...
893,The Villa Rose Hotel & V-Spa,"Main Street, Ballybofey, Ireland",8.9,Fabulous,"-7.78147995471954,54.7995469321867",54.7995469321867,-7.78147995471954,https://booking.com/hotel/ie/the-villa-rose-v-...
894,Neidin House,"Landsdown Lodge, Kenmare, Ireland",9.0,Superb,"-9.57696944475174,51.880729836645",51.880729836645,-9.57696944475174,https://booking.com/hotel/ie/neidin-house.en-g...
895,Hillview Self Catering,"Ballaghstown, Lusk, Lusk, Ireland",9.6,Exceptional,"-6.14556312561035,53.5423826268545",53.5423826268545,-6.14556312561035,https://booking.com/hotel/ie/hillview-self-cat...


## Work in Progress

Scraping the table from booking.com hotel data

In [20]:
url_room_1 = "https://www.booking.com/hotel/ie/kilcoran-lodge-leisure-centre.en-gb.html?label=gen173nr-1FCAEoggI46AdIM1gEaGmIAQGYAQm4ARfIAQzYAQHoAQH4AQuIAgGoAgO4AvLEr4gGwAIB0gIkYmNjMGYwYTItZTY1MS00Zjc2LWFmN2YtOTY4MjRiODNiZmYz2AIG4AIB;sid=11459958cab5ba31275092207994b678;all_sr_blocks=27039103_232331336_0_2_0;checkin=2021-10-09;checkout=2021-10-21;dest_id=102;dest_type=country;dist=0;group_adults=2;group_children=0;hapos=1;highlighted_blocks=27039103_232331336_0_2_0;hpos=1;no_rooms=1;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=27039103_232331336_0_2_0__146999;srepoch=1628695588;srpvid=25686c91d15e019c;type=total;ucfs=1&#hotelTmpl"
html = urlopen(url_room_1)
response = requests.get(url_room_1, headers=headers)
soup_link = BeautifulSoup(response.content, 'lxml')
room_types = []
prices = []


for item_1 in soup_link.select('.hprt-container'):
    room = item_1.select('.hprt-roomtype-icon-link')[0].get_text().strip()
    print(room)
    room_types.append(room)
    price = item_1.select('.bui-u-sr-only')[0].get_text().strip()
    price = price.replace('Price', '').strip()
    price = price.replace(u'\xa0', u'')
    price = price.replace('€','')
    price = price.replace(',','')
    

    prices.append(price.strip())
#     print(item.select('.hprt-roomtype-icon-link')[0].get_text().strip())

# for item_2 in soup_link.select('.d_pd_hp_price_left_align'):
#     price = item_2.select('.bui-u-sr-only')[0].get_text().strip()
#     price = price.replace('Price', '').strip()
#     price = price.replace(u'\xa0', u'')
#     price = price.replace('€','')
#     price = price.replace(',','')
    

#     prices.append(price.strip())


rooms = " ".join(str(elem) for elem in room_types)
price_new = " ".join(str(elem) for elem in prices)

print(rooms)
print(price_new)

Double Room
Double Room
Max persons: 2
